# Content-Based Recommender

准备每部电影的 特征向量 x^(i)

In [1]:
import numpy as np

# 假设我们有3部电影，每部电影的特征是 [浪漫, 动作, 科幻]
X = np.array([
    [0.8, 0.1, 0.1],  # Movie A
    [0.0, 1.0, 0.0],  # Movie B
    [0.2, 0.2, 0.6]   # Movie C
])


为每个用户准备一个 兴趣向量 θ^(j)

In [2]:
theta_alice = np.array([5.0, 0.0, 0.0])  # 她只喜欢浪漫片


用点积预测评分 y^(i,j) = (θ^(j))^T x^(i)

In [4]:
predicted_ratings = X @ theta_alice  # 点积操作
print(predicted_ratings)  # 输出这三部电影的预测评分


[4. 0. 1.]


要“学的是 θ”，而不是“假设已有 θ”

用已有评分 y 来优化 θ，使得 (θ^T x) 尽可能接近 y

实现损失函数 J(θ)

推出梯度 ∇J(θ)

编写 theta 的迭代更新代码

In [10]:
theta_alice = np.random.randn(3)
y = np.array([4.0, 1.0, 2.0])  # Alice 对三部电影的真实评分


In [11]:
def loss_function(theta, X, y):
    predictions = X @ theta
    return np.mean((predictions - y) ** 2)

In [12]:
def gradient(theta, X, y):
    predictions = X @ theta
    errors = predictions - y
    gradient = X.T @ errors / len(y)
    return gradient

In [15]:
step = 100
learning_rate = 0.01
for i in range(step):
    gradient_theta = gradient(theta_alice, X, y)
    theta_alice -= learning_rate * gradient_theta

    # 打印损失(每十步)
    if i % 10 == 0:
      current_loss = loss_function(theta_alice, X, y)
      print(f"Step {i}, Loss: {current_loss:.4f}")

Step 0, Loss: 12.0655
Step 10, Loss: 11.3324
Step 20, Loss: 10.6462
Step 30, Loss: 10.0038
Step 40, Loss: 9.4023
Step 50, Loss: 8.8390
Step 60, Loss: 8.3113
Step 70, Loss: 7.8169
Step 80, Loss: 7.3537
Step 90, Loss: 6.9194


从原来一个人对若干电影的评分数据，扩展成一个**用户-评分矩阵**结构。
关键设计变成了：

1. 输入结构：

  - X：每部电影的特征（不变）
  - Y：每个用户对每部电影的评分矩阵（比如 shape 为 m × n）
  - R：一个同样 shape 的 0/1 矩阵，表示用户是否给这部电影打过分（mask）

2. 参数：

  每个用户都有一个 θ^(j)，所以我们现在需要一个 theta_all，shape 是 n × k（n个用户，k个特征）

3. 训练：

  - 遍历每个用户 j：

  - 取出 j 用户有评分的电影行

  - 用这些电影的特征和评分训练 θ^(j)

  - 存入 theta_all[j]

准备数据结构（多人版）

备注: 掩码矩阵
```
array([
    [False, False, False],
    [False,  True, False]
])
```
再取反:
```
array([
    [ True,  True,  True],
    [ True, False,  True]
])

```

In [18]:
# 3部电影，每部特征是 [浪漫, 动作, 科幻]
X = np.array([
    [0.8, 0.1, 0.1],  # Movie A
    [0.0, 1.0, 0.0],  # Movie B
    [0.2, 0.2, 0.6]   # Movie C
])

# 2个用户对3部电影的评分（NaN 表示没评分）
Y = np.array([
    [4.0, 1.0, 2.0],     # Alice
    [5.0, np.nan, 1.0]   # Bob
])

# 构建 R 掩码矩阵：用户是否对该电影打过分
R = ~np.isnan(Y)  # True if rated


为每个用户训练 θ^(j)
我们现在要循环每个用户,选出她打过分的电影, 然后单独训练她的向量

In [19]:
theta_all = [] # 收集所有兴趣向量

# 我们开始对每个用户编号，依次叫他们进来训练。假设有 2 个用户，这个 user_id 就会是 0 和 1。
for user_id in range(Y.shape[0]):
    # 找出当前用户打过分的电影
    idx = R[user_id]
    X_user = X[idx]           # 这些电影的特征
    y_user = Y[user_id][idx]  # 对应评分

    # 初始化 θ^(j)
    theta = np.random.randn(X.shape[1])

    # 梯度下降训练 θ^(j)
    for step in range(100):
        pred = X_user @ theta
        error = pred - y_user
        grad = X_user.T @ error / len(y_user)
        theta -= 0.01 * grad

    theta_all.append(theta)


把训练好的 theta_all 用来“预测”每个用户对所有电影的评分

In [20]:
import numpy as np

theta_all = np.array(theta_all)         # 列表转成矩阵
predicted_all = theta_all @ X.T         # 每个用户对所有电影的预测评分

print("预测评分矩阵：")
print(predicted_all)


预测评分矩阵：
[[ 0.31989075  1.56290138  0.77025802]
 [ 1.64955014 -1.17719668  0.52611255]]


# Collaborative Filtering

和 content-based 最大不同是：现在 x^(i) 不是事先定义的特征向量，而是也要通过训练学习

1. 首先做矩阵
我们会用这个评分矩阵同时训练：

theta_all：每个用户的兴趣向量，shape 是 (n_users × k)

x_all：每部电影的隐特征向量，shape 是 (n_movies × k)

(k是特征的数量 -- 隐变量的维度)

In [21]:
# 超简化的评分矩阵(同上)
Y = np.array([
    [5, 4, np.nan],  # Alice
    [np.nan, 3, 1],  # Bob
    [2, np.nan, 4]   # Carol
])

R = ~np.isnan(Y)  # 是否评分的掩码矩阵


2. 全部随机初始化参数


In [22]:
n_users, n_movies = Y.shape
k = 3  # 隐变量维度

theta_all = np.random.randn(n_users, k)
x_all = np.random.randn(n_movies, k)


3. 定义损失函数（含正则化）

In [23]:
def loss_function(theta_all, x_all, Y, R, λ):
    loss = 0
    for i in range(n_users):
        for j in range(n_movies):
            if R[i][j]:
                pred = theta_all[i] @ x_all[j]
                loss += (pred - Y[i][j]) ** 2
    loss *= 0.5
    loss += (λ / 2) * (np.sum(theta_all ** 2) + np.sum(x_all ** 2))
    return loss


4. 梯度下降 theta 和 x

In [24]:
def compute_theta_gradient(i, theta_all, x_all, Y, R, λ):
  grad = np.zeros_like(theta_all[i])  # 初始梯度是全零
  for j in range(x_all.shape[0]):
    if R[i][j]:
      pred = theta_all[i] @ x_all[j]

      error = pred - Y[i][j]
      grad += error * x_all[j]
  grad += λ * theta_all[i]
  return grad


In [25]:
def compute_x_gradient(j, theta_all, x_all, Y, R, λ):
    grad = np.zeros_like(x_all[j])
    for i in range(theta_all.shape[0]):
        if R[i][j]:
            pred = theta_all[i] @ x_all[j]
            error = pred - Y[i][j]
            grad += error * theta_all[i]
    grad += λ * x_all[j]
    return grad


In [26]:
def train(theta_all, x_all, Y, R, λ, learning_rate, steps):
    for step in range(steps):
        # 更新所有 θ⁽ⁱ⁾
        for i in range(theta_all.shape[0]):
            grad_theta = compute_theta_gradient(i, theta_all, x_all, Y, R, λ)
            theta_all[i] -= learning_rate * grad_theta

        # 更新所有 x⁽ʲ⁾
        for j in range(x_all.shape[0]):
            grad_x = compute_x_gradient(j, theta_all, x_all, Y, R, λ)
            x_all[j] -= learning_rate * grad_x

        # 每10步打印一次 loss
        if step % 10 == 0:
            loss = loss_function(theta_all, x_all, Y, R, λ)
            print(f"Step {step} | Loss: {loss:.4f}")


In [27]:
train(theta_all, x_all, Y, R, λ=0.1, learning_rate=0.01, steps=100)


Step 0 | Loss: 36.9784
Step 10 | Loss: 11.9560
Step 20 | Loss: 5.2829
Step 30 | Loss: 3.8392
Step 40 | Loss: 3.3513
Step 50 | Loss: 3.0418
Step 60 | Loss: 2.7883
Step 70 | Loss: 2.5635
Step 80 | Loss: 2.3579
Step 90 | Loss: 2.1688
